In [29]:
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import datetime

In [30]:
# load data_set

transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 缩放到224 * 224
    # transforms.CenterCrop(256)   #中心剪裁后四周padding补充 (后续可以padding)
     transforms.ToTensor()
#     transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])  # 均值为0 方差为1 的正态分布
])

# 0 -> mask  1-> nonmask  2 -> not a person
train_dataset = ImageFolder('./data/train', transform=transform)

# for img, label in train_dataset:
#     if (label == 0):
#         print(img.shape)

# # train 每四个为一组
# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# # 显示图片
# to_pil_image = transforms.ToPILImage()
# for image, label in train_dataloader:
#     img = to_pil_image(image[0])
#     img.show()

In [31]:
# # calculate mean of imgs in each RGB channel
# imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
# print(imgs.shape)

# imgs.view(3, -1).mean(dim=1)  # mean of (number of imgs) in each channel

In [32]:
# imgs.view(3, -1).std(dim=1) # std of (number of imgs) in each channel

In [33]:
# load dataset, normarlize it
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 缩放到224 * 224
    # transforms.CenterCrop(256)   #中心剪裁后四周padding补充 (后续可以padding)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4990, 0.4567, 0.4188], std=[0.2913, 0.2778, 0.2836]) 
])

# 0 -> mask  1-> nonmask  2 -> not a person
train_dataset = ImageFolder('./data/train', transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=50, shuffle=True)

In [34]:
# for img, label in train_dataset:
#     print(img.shape)
#     print(label)

In [35]:
# for imgs, labels in train_dataloader:
#     print(imgs.shape)
#     print(labels.shape)

In [36]:
# cnn model

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), # in=3x256x256; out=32x256x256
            nn.Tanh(),
            nn.MaxPool2d(2), # out=64x128x128
            nn.Conv2d(32, 16, kernel_size=3, padding=1), # in=32x128x128, out=16x128x128
            nn.Tanh(),
            nn.MaxPool2d(2), # out=16x64x64
            nn.Conv2d(16, 8, kernel_size=3, padding=1), # in=16x64x64, out=8x64x64
            nn.Tanh(),
            nn.MaxPool2d(2) # out=8x32x32
        )
        
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(8*32*32, 32*32),
            nn.Tanh(),
            nn.Dropout(p=0.1),
            nn.Linear(32*32, 1024),
            nn.Tanh(),
            nn.Dropout(p=0.1),
            nn.Linear(1024, 128),
            nn.Tanh(),
            nn.Dropout(p=0.1),
            nn.Linear(128, 3)
        )
    
    def forward(self, x):
        # conv layer
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)
        
        return x


In [37]:
device = (torch.device('cuda') if torch.cuda.is_available() 
         else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [38]:
# define train_loop function
def train_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_dataloader: # loop over batches in dataset
            # move data to GPU if available
            imgs = imgs.to(device=device)  
            labels = labels.to(device=device)
            
            outputs = model(imgs)  # feed a batch through our model
            
            loss = loss_fn(outputs, labels)  # computes the loss
            
            optimizer.zero_grad()  # getting rid of the gradients from the last round
            
            loss.backward()  # performs backward step, compute the gradients of all parameters
            
            optimizer.step()  # updates the model
            
            loss_train += loss.item() # sums of losses we saw over the epoch
            
        # print the average loss per batch, in epoch%10 == 0 
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch, loss_train/len(train_loader)
            ))
            

In [39]:
model = CNN().to(device=device)  # instantiates cnn model

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(9577499,
 [864,
  32,
  4608,
  16,
  1152,
  8,
  8388608,
  1024,
  1048576,
  1024,
  131072,
  128,
  384,
  3])

In [40]:
# perform training, overfitting on small sample data with large amount of parameters

learning_rate = 0.02
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()  # use cross entropy loss function

# call train_loop() function
train_loop(
    n_epochs = 30,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_dataloader
)

2021-03-16 19:00:21.002236 Epoch 1, Training loss 1.0474044110464014
2021-03-16 19:02:18.439236 Epoch 10, Training loss 0.545487440150717
2021-03-16 19:04:28.858236 Epoch 20, Training loss 0.3382554274538289
2021-03-16 19:06:39.218236 Epoch 30, Training loss 0.14587856424243553


In [41]:
# define validate function
def validate(model, train_loader, test_loader):
    model.eval()
    # accuracy on training data and test data
    for name, loader in [("train", train_loader), ("test", test_loader)]:
        correct = 0
        total = 0
        
        with torch.no_grad(): # do not want gradients here, as we will not want to update parameters
            for imgs, labels in loader:
                # move data to GPU if available
                imgs = imgs.to(device=device)  
                labels = labels.to(device=device)
                
                outputs = model(imgs)  # feed input to models
                
                _, predicted = torch.max(outputs, dim=1)  # gives the index of the highest value as output
                
                total += labels.shape[0]  # counts the number of example, total is increased by the batch size
                
                # comparing the predicted class that had the maximum probability and the ground-truth labels,
                # we first get a Boolean array. Taking the sum gives the number of items in the batch where 
                # the prediction and ground truth agree
                correct += int((predicted == labels).sum()) 
        
        print("Accuracy {}: {:.2f}".format(name, correct / total))
                

In [42]:
# measuring accuracy
test_dataset = ImageFolder('./data/test', transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

validate(model, train_dataloader, test_dataloader)


Accuracy train: 0.97
Accuracy test: 0.73
